In [ ]:
# 20JUN21, SUN
# [Public] Location:  https://drive.google.com/drive/folders/1fFoFxY4i3YU-KNkz84_AG9Nftxr4Alnb

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7MB 4.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.3MB 187kB/s 
     |████████████████████████████████| 9.9MB 177kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=c7688d48eb669ad22bba4cc5a0b784c560b29334f6812bce683b9780aac4301c
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.

Click &nbsp; 'RESTART RUNTIME', &nbsp; and then &nbsp; 'Yes'

In [1]:
cd "gdrive/MyDrive/Time Magazine"

/content/gdrive/MyDrive/Time Magazine


In [2]:
import re, os, ast, time, requests, urllib.request
from datetime import datetime
from pytz import timezone


!sudo apt install  tesseract-ocr
!pip install pytesseract

import pytesseract
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract, cv2, os
import cv2, random, itertools
from google.colab.patches import cv2_imshow
import numpy as np
from sklearn.cluster import DBSCAN


!python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load('en_core_web_sm')

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stopword_set = set(stopwords.words('english'))


from __future__ import print_function

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (4,461 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


# Scraping [Time Magazine](https://time.com/vault/)
1923 - 2015

In [ ]:
def get_issue_dates(year):
    url = 'https://time.com/vault/year/{}/'.format(year)
    page = requests.get(url)
    issue_dates  =  re.findall('\"coverDate\":\"(\d\d\d\d-\d\d-\d\d)\"', page.text)
    return issue_dates

def download_page_image(url, path):
    try:
        urllib.request.urlretrieve(url, path)
        return True
    except Exception as e:
        print('\n\tURL:', url, '\n\tPath:', path)
        print('\t', e)
        return False

def download_issue(issue_date):
    year, month, day  =  issue_date.split('-')
    
    # Find Number of Pages
    url = 'https://time.com/vault/issue/{}/page/1/'.format(issue_date)
    page = requests.get(url)
    page_count = len(ast.literal_eval(re.findall('\"pagePDFs\":(\[[^\[\]]*\])', page.text)[0]))

    # Create Issue Directory
    if not os.path.exists(year+'/'+issue_date):    os.mkdir(year+'/'+issue_date)

    # Download Pages
    for  page_number  in range(1, page_count+1):
        page_url  =  'https://content.time.com/time/subscriber/vault/{}/{}/{}/{}/1550.jpg'.format(year, month, year+month+day, page_number)
        
        # urllib.request.urlretrieve(page_url, '{}/{}/{}.jpg'.format(year, issue_date, page_number))
        count = 0
        success  =  download_page_image(page_url, '{}/{}/{}.jpg'.format(year, issue_date, page_number))
        while not success and count < 3:
            success  =  download_page_image(page_url, '{}/{}/{}.jpg'.format(year, issue_date, page_number))
            time.sleep(1)
            count += 1


def get_timestamp():
    now_utc = datetime.now(timezone('UTC'))
    return now_utc.astimezone(timezone('Asia/Kolkata')).strftime("[%d-%m-%Y %H:%M:%S]")


In [ ]:
%%time


for  year  in list(range(1923, 2021+1)):
    year = str(year)
    
    # Create year Directory
    if not os.path.exists(year):    os.mkdir(year)

    issue_dates  =  get_issue_dates(year)
    for  issue_date  in issue_dates:
        download_issue(issue_date)


    print('\n{}  Completed: {}.'.format(get_timestamp(), year))

# Text Extracation (2000-2015)

In [4]:
for  year  in  range(2000, 2016):
    year = str(year)
    
    for  path,subdirs,files  in os.walk(year):
        dir_files = os.listdir(path)
        print(path)
        
        for name in files:
            if name.endswith('.jpg')  and  name[:-4]+'.txt' not in dir_files:
                img_path = os.path.join(path, name)

                image = cv2.imread(img_path)
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)    # Convert to Black and White
                gray = cv2.resize(gray, None, fx = 3, fy = 3, interpolation = cv2.INTER_CUBIC)    # Zoom 3X

                text  =  pytesseract.image_to_string(gray)    # OCR

                # Save text
                f = open(img_path[:-4]+'.txt', 'w')
                f.write(text)
                f.close()

2000
2000/2000-01-01
2000/2000-01-17
2000/2000-01-24
2000/2000-01-31
2000/2000-02-07
2000/2000-02-14
2000/2000-02-21
2000/2000-02-28
2000/2000-03-06
2000/2000-03-13
2000/2000-03-20
2000/2000-03-27
2000/2000-04-03
2000/2000-04-10
2000/2000-04-17
2000/2000-04-24
2000/2000-05-01
2000/2000-05-08
2000/2000-05-15
2000/2000-05-22
2000/2000-05-29
2000/2000-06-05
2000/2000-06-12
2000/2000-06-19
2000/2000-06-26
2000/2000-07-03
2000/2000-07-10
2000/2000-07-17
2000/2000-07-24
2000/2000-07-31
2000/2000-08-07
2000/2000-08-14
2000/2000-08-21
2000/2000-08-28
2000/2000-09-04
2000/2000-09-11
2000/2000-09-18
2000/2000-09-25
2000/2000-10-02
2000/2000-10-09
2000/2000-10-16
2000/2000-10-23
2000/2000-10-30
2000/2000-11-06
2000/2000-11-13
2000/2000-11-20
2000/2000-11-27
2000/2000-12-04
2000/2000-12-11
2000/2000-12-18
2000/2000-12-25
2001
2001/2001-01-08
2001/2001-01-15
2001/2001-01-22
2001/2001-01-29
2001/2001-02-05
2001/2001-02-12
2001/2001-02-19
2001/2001-02-26
2001/2001-03-05
2001/2001-03-12
2001/2001-03-1

# What is on the cover page?
The most Frequent Terms

In [6]:
try:    from unidecode import unidecode
except:
    !pip install unidecode
    from unidecode import unidecode


def read_text_file(filename):
    with open(filename) as f:    
        content = ' '.join([unidecode(i.strip())   for i in f.readlines()   if i.strip()]).replace('- ', '')
    return content

def get_magazine_text(path):
    pages  =  sorted([os.path.join(path, file)   for file in os.listdir(path)   if file.endswith('.txt')], key=lambda x: int(x.split('/')[-1][:-4]))
    texts  =  [read_text_file(page)  for page in pages]
    return texts


     |████████████████████████████████| 245kB 3.9MB/s 


In [3]:
release_dates  =  sum([[os.path.join(str(year), dir_)  for dir_ in os.listdir(str(year))  if '.' not in dir_]  for year in range(2000, 2016)], [])


In [7]:
texts = []
for  release_date  in release_dates:
    # print('\n\n\n', release_date)
    text = read_text_file(release_date+'/1.txt')
    texts.append(text)
    # print(text)

    # doc = nlp(text)
    # for entity in doc.ents:
    #     print(entity.label_, ' | ', entity.text)

In [8]:
from collections import Counter
l = ' '.join(texts).lower().split()
l = [i  for i in l  if len(i)>=3 and i not in stopword_set]
c = Counter(l)
c.most_common()

[('new', 75),
 ('joe', 57),
 ('eee', 56),
 ('time', 55),
 ('special', 49),
 ('inside', 39),
 ('michael', 39),
 ('klein', 38),
 ('america', 33),
 ('world', 28),
 ('plus:', 28),
 ('obama', 28),
 ('report', 27),
 ('david', 27),
 ('see', 24),
 ('health', 24),
 ('page', 24),
 ('one', 23),
 ('eae', 22),
 ('u.s.', 22),
 ('make', 21),
 ('war', 21),
 ('issue', 20),
 ('von', 19),
 ('drehle', 18),
 ('science', 17),
 ('behind', 17),
 ('look', 17),
 ('best', 16),
 ("america's", 16),
 ('barack', 16),
 ("what's", 15),
 ('bush', 15),
 ('way', 15),
 ('real', 15),
 ('exclusive', 15),
 ('president', 15),
 ('klein:', 15),
 ('john', 14),
 ('(c)', 14),
 ('change', 14),
 ('want', 14),
 ('care', 14),
 ('like', 13),
 ('could', 13),
 ('really', 13),
 ('american', 13),
 ("he's", 13),
 ('save', 13),
 ('fareed', 13),
 ('future', 12),
 ('still', 12),
 ('aaa', 12),
 ('life', 12),
 ('need', 12),
 ('guide', 12),
 ('election', 12),
 ('economy', 12),
 ("obama's", 12),
 ('battle', 11),
 ('case', 11),
 ('politics', 11),
 

### Looks like it's mostly about the US and Geopolitics.

In [ ]:
# Save all texts in a Pickle File (for quick retreval).
# texts = {}
# count = 0

# for  release_date  in release_dates:
#     print(release_date)
#     for  file_  in os.listdir(release_date):
#         count += 1
#         texts[os.path.join(release_date, file_)]  =  (count, get_magazine_text(release_date))


# import pickle
# with open('time_magazine_texts.pickle', 'wb') as f:
#     pickle.dump(texts, f, protocol=pickle.HIGHEST_PROTOCOL)

# with open('time_magazine_texts.pickle', 'rb') as f:
#     texts = pickle.load(f)


2000/2000-01-01
2000/2000-01-17
2000/2000-01-24
2000/2000-01-31
2000/2000-02-07
2000/2000-02-14
2000/2000-02-21
2000/2000-02-28
2000/2000-03-06
2000/2000-03-13
2000/2000-03-20
2000/2000-03-27
2000/2000-04-03
2000/2000-04-10
2000/2000-04-17
2000/2000-04-24
2000/2000-05-01
2000/2000-05-08
2000/2000-05-15
2000/2000-05-22
2000/2000-05-29
2000/2000-06-05
2000/2000-06-12
2000/2000-06-19
2000/2000-06-26
2000/2000-07-03
2000/2000-07-10
2000/2000-07-17
2000/2000-07-24
2000/2000-07-31
2000/2000-08-07
2000/2000-08-14
2000/2000-08-21
2000/2000-08-28
2000/2000-09-04
2000/2000-09-11
2000/2000-09-18
2000/2000-09-25
2000/2000-10-02
2000/2000-10-09
2000/2000-10-16
2000/2000-10-23
2000/2000-10-30
2000/2000-11-06
2000/2000-11-13
2000/2000-11-20
2000/2000-11-27
2000/2000-12-04
2000/2000-12-11
2000/2000-12-18
2000/2000-12-25
2001/2001-01-08
2001/2001-01-15
2001/2001-01-22
2001/2001-01-29
2001/2001-02-05
2001/2001-02-12
2001/2001-02-19
2001/2001-02-26
2001/2001-03-05
2001/2001-03-12
2001/2001-03-19
2001/200

# Let's try to find some topics and their dominant terms

In [4]:
len(release_dates)

652

In [ ]:
# Let's see for a year first.

In [7]:
%%time
l = [get_magazine_text(release_date)   for release_date in release_dates[:1]]
pages = [item  for sublist in l  for item in sublist]

CPU times: user 182 ms, sys: 34.2 ms, total: 217 ms
Wall time: 21.8 s


In [8]:
n_components = 5    # Number of topics

In [9]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(pages)
print(dtm_tf.shape)


# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=n_components, random_state=0)
lda_tf.fit(dtm_tf)


pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

(126, 173)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.156391  0.203742       1        1  33.892033
1      0.162202 -0.000506       2        1  26.958816
4      0.061638  0.002604       3        1  17.014650
3      0.123062 -0.023692       4        1  14.583641
2     -0.190510 -0.182146       5        1   7.550860, topic_info=         Term        Freq       Total Category  logprob  loglift
18        com   69.000000   69.000000  Default  30.0000  30.0000
31     doctor  100.000000  100.000000  Default  29.0000  29.0000
101       new  121.000000  121.000000  Default  28.0000  28.0000
167       www   30.000000   30.000000  Default  27.0000  27.0000
142    taking   69.000000   69.000000  Default  26.0000  26.0000
..        ...         ...         ...      ...      ...      ...
36        eee    3.806347   37.512273   Topic5  -4.4680   0.2955
2     america    3.083395   27.601856   Topic5  -4.6787   0.3917
3    american    3.264879   36.322375   Topic5  -4.6215   0.1743
154       use    3.229740   52.444997   Topic5  -4.6323  -0.2039
69       just    3.134603   42.972982   Topic5  -4.6622  -0.0346

[228 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.307484    age
0         3  0.345919    age
0         4  0.230613    age
0         5  0.076871    age
1         2  0.208376    ago
...     ...       ...    ...
171       3  0.083424   york
172       1  0.061168  young
172       2  0.183505  young
172       3  0.489347  young
172       4  0.244674  young

[458 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 5, 4, 3])

In [10]:
tfidf_vectorizer = TfidfVectorizer(strip_accents = 'unicode',
                                   stop_words = 'english',
                                   lowercase = True,
                                   token_pattern = r'\b[a-zA-Z]{3,}\b',
                                   max_df = 0.5, 
                                   min_df = 10)
dtm_tfidf = tfidf_vectorizer.fit_transform(pages)
print(dtm_tfidf.shape)


# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=n_components, random_state=0)
lda_tfidf.fit(dtm_tfidf)


pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

(126, 173)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.095940 -0.021573       1        1  63.219038
0     -0.134004 -0.009077       2        1  19.098246
2      0.020219 -0.037217       3        1   7.093561
1      0.021653  0.073219       4        1   6.140158
4     -0.003808 -0.005352       5        1   4.448998, topic_info=         Term      Freq     Total Category  logprob  loglift
18        com  6.000000  6.000000  Default  30.0000  30.0000
170   yeltsin  3.000000  3.000000  Default  29.0000  29.0000
167       www  4.000000  4.000000  Default  28.0000  28.0000
31     doctor  4.000000  4.000000  Default  27.0000  27.0000
7        best  3.000000  3.000000  Default  26.0000  26.0000
..        ...       ...       ...      ...      ...      ...
3    american  0.100313  3.756085   Topic5  -5.2361  -0.5103
57       help  0.100313  3.103378   Topic5  -5.2361  -0.3195
102     night  0.100313  2.419845   Topic5  -5.2361  -0.0707
144     thing  0.100312  2.095279   Topic5  -5.2361   0.0733
71       know  0.100310  2.939769   Topic5  -5.2361  -0.2653

[238 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1         1  0.798686        ago
2         1  0.777537    america
3         1  0.798704   american
4         2  0.572206        ask
5         1  0.353731  available
...     ...       ...        ...
168       1  0.861030       year
168       4  0.172206       year
169       1  0.802490      years
170       4  0.971691    yeltsin
171       1  0.692386       york

[162 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])

In [ ]:
# TODO: For all the issues, and interpret 